##### CSCI 303
# Introduction to Data Science
<p/>
### 11 - Data Acquisition




## This Lecture
---
- Importing data using Python and pandas

The obligatory setup code...

In [1]:
import numpy as np
import pandas as pd

from pandas import Series, DataFrame

## Raw Python
---
Text-based files are easy to read and write in Python.

In particular, files which data is organized into individual lines.

There's a sample file, `text.txt`, located in the directory `11-acquisition-files`.

We can view it by asking Jupyter to execute the linux command `cat` on the file (only works if you are not using Windows):

In [1]:
!cat 11-acquisition-files/text.txt

'cat' is not recognized as an internal or external command,
operable program or batch file.


Reading the file via Python is pretty simple (use this if you are using a Windows operating system):

In [5]:
f = open('11-acquisition-files/text.txt') # get an open file object
for line in f:                            # for loops work!
    print(line, end='')                   # print without an endline

f.close()

FileNotFoundError: [Errno 2] No such file or directory: '11-acquisition-files/text.txt'

Python has many tools to let us relatively easily parse files like this.

For simple files, the string `split` method may suffice:

In [ ]:
f = open('11-acquisition-files/text.txt')
for line in f:
    print(line.split())
f.close()

If you know regular expressions, you can parse more complicated files.

However, you're unlikely to need something that complicated.

Let's bring this data into pandas, the hard way:

In [ ]:
data = []
f = open('11-acquisition-files/text.txt')
columns = next(f).split()
for line in f:
    data.append(line.split())
f.close()
df = DataFrame(data=data, columns=columns)
df

## pandas `read_table`
---
For many file formats, it is easiest to let pandas do all the work!

One of the most basic tools is used to read text files like the one we worked with above is `read_table`.

In [ ]:
pd.read_table?

We don't need to do much for this file, as it is nicely tab-delimited and has a header row.

In [ ]:
df2 = pd.read_table('11-acquisition-files/text.txt')
df2

Our table might not have a header row, in which case we have to supply column labels:

In [ ]:
df3 = pd.read_table('11-acquisition-files/text-no-header.txt', 
                    header=None, names=['Person','Age','Income'])
df3

There are lots of other ways to manipulate the data when reading.

For instance, we can make one column the row index:

In [ ]:
df4 = pd.read_table('11-acquisition-files/text.txt', 
                    index_col=0)
df4

We can also do this by giving the column header:

In [ ]:
df5 = pd.read_table('11-acquisition-files/text.txt', 
                    index_col='Name')
df5

If tabs are not your separator, you can specify separators either as a specific character, or using a regular expression.

For arbitrary length whitespace separation, use `'\s+'`:

In [ ]:
df6 = pd.read_table('11-acquisition-files/text.txt', sep='\s+')
df6

Don't want the whole thing?

Check out the `nrows` and `skiprows` arguments:

In [ ]:
df7 = pd.read_table('11-acquisition-files/text.txt', nrows=3)
df7

In [ ]:
df8 = pd.read_table('11-acquisition-files/text.txt', 
                    skiprows=[0,1])
df8

## Type Inference in `read_table`
---
Let's take a closer look at the DataFrame objects we're getting back:

In [ ]:
df9 = pd.read_table('11-acquisition-files/text.txt')
print(df9.info())
df9

It correctly *inferred* the type of the Age column as an integer.

But it didn't pick up Salary as a number, which we kind of want.

OK, this can be fixed!

In [ ]:
df10 = pd.read_table('11-acquisition-files/text.txt', 
                     thousands=',')
df10.info()
#pd.read_table?

Dates are also something pandas knows about:

In [ ]:
df11 = pd.read_table('11-acquisition-files/text.txt', 
                     thousands=',', 
                     parse_dates=['Hired'])
df11.info()

The display data now looks slightly different:

In [ ]:
df11

## pandas `read_csv`
---
A startling amount of the time, data is available in *comma-separated values* format.

The CSV format is commonly used to exchange data between things like spreadsheets and databases.

Here's what our data might look like exported from a spreadsheet program:

In [ ]:
!cat 11-acquisition-files/text.csv

pandas `read_csv` is pretty much identical to `read_table`, but assumes a CSV format:

In [ ]:
df12 = pd.read_csv('11-acquisition-files/text.csv', 
                   parse_dates=['Hired'])
df12

## JSON Data
---
Increasingly data is available in JSON (*JavaScript Object Notation*) format.

JSON is a format for exchanging rich, structured data as plain text.

JSON object strings look (and act) remarkably like Python code for dictionaries:

{ "Name" : "Laura",
  "Age" : 52,
  "Salary" : 103790,
  "Pets" : [ 
      { "type" : "rabbit", "name" : "Gandalf" },
      { "type" : "dog", "name" : "Aragorn" }
  ]
}


JSON basic types include strings, numbers (integer or floating point), Booleans, and nulls.

Compound types are objects and lists, which correspond pretty directly to Python dicts and lists.

Python has a library for interpreting JSON strings and turning them into Python objects (and vice versa):


In [ ]:
import json

s = '''
   { "Name" : "Laura", 
     "Age" : 52, 
     "Salary" : 103790, 
     "Pets" : [ 
       { "type" : "rabbit", "name" : "Gandalf" }, 
       { "type" : "dog", "name" : "Aragorn" } ] }
    '''

obj = json.loads(s)
obj

It turns out that pandas DataFrame objects can be created from Python dictionaries, although its approach isn't always exactly what you expect:

In [ ]:
df13 = DataFrame(obj)
df13

When dictionaries are fairly "flat", pandas interprets them pretty sensibly.

The two structures it handles best are dictionaries where the keys represent columns, and lists, where the entries are dictionaries representing rows.

Here's an example for the first:

In [ ]:
d =  {"Name"   : ["Laura","Shashi","Jun","Bruce","Raluca"],
      "Age"    : [52,46,33,48,40],
      "Salary" : [103790,89100,85500,96445,110080],
      "Hired"  : ["1/1/2005","6/16/2010","7/1/2017","12/1/2008","9/15/2012"]}
DataFrame(d)

and the second:

In [ ]:
d2 = [
    {"Name" : "Laura", "Age" : 52, "Salary" : 103790, "Hired" : "1/1/2005"},
    {"Name" : "Shashi", "Age" : 46, "Salary" : 89100, "Hired" : "6/16/2010"},
    {"Name" : "Jun", "Age" : 33, "Salary" : 85500, "Hired" : "7/1/2017"},
    {"Name" : "Bruce", "Age" : 48, "Salary" : 96445, "Hired" : "12/1/2008"},
    {"Name" : "Raluca", "Age" : 40, "Salary" : 110080, "Hired" : "9/15/2012"}
]
DataFrame(d2)

In both cases, pandas re-ordered the columns alphabetically; we can tell it what order we want things in:

In [ ]:
DataFrame(d2, columns=['Name','Age','Salary','Hired'])

Additional structures are possible, such as ones in which row index labels are explicitly provided:

In [ ]:
d3 =  {"Age"    : {"Laura" : 52, "Shashi" : 46},
       "Salary" : {"Laura" : 103790, "Shashi" : 89100},
       "Hired"  : {"Laura" : "1/1/2005", "Shashi" : "6/16/2010"}}
DataFrame(d3)

Not surprisingly, then, if data is stored in JSON in any of these structures, pandas can read it quite easily.

Here is our simple data set in JSON data formats:

In [ ]:
!cat 11-acquisition-files/json1.json
d14 = pd.read_json('11-acquisition-files/json1.json')
d14

In [ ]:
!cat 11-acquisition-files/json2.json
d15 = pd.read_json('11-acquisition-files/json2.json', 
                   convert_dates=['Hired'])
d15

Oddly, there doesn't seem to be a way to set the order of columns in `read_json`, but we can reorder pretty easily:

In [ ]:
d16 = d15[['Name','Age','Salary','Hired']]
d16